In [2]:
import sys
import json
import time
import numpy as np
import pinocchio as pin
import matplotlib.pyplot as plt
sys.path.insert(0, '..')
from src.embedding import Embedding, Collision, JointLimit
from src.forward_kinematics import ForwardKinematic
from src.dynamical_system import DynamicalSystem
from utils import franka_parameters
from utils.visualization import plot_3d_ellipsoid_from_covariance

In [3]:
with open('../config/environment4.json') as file:
    config = json.load(file)

In [ ]:
# add an obstacle to the workspace
x = np.array(config['obstacles'])
# place an attractor in the configuration space
config_attractor = np.array([a * np.pi/180 for a in config['attractor']])

fk = ForwardKinematic(
    urdf_file=config['urdf'],
    gmm_configuration_file="../config/gmm_unit.json"
)

e = Embedding(embeddings=[Collision(x=x, fk=fk), JointLimit(limits=franka_parameters.joint_limits, booster=10)])

# define proportional and dissipation gains for the DS
k, d = 0.5, 1.5
ds = DynamicalSystem(stiffness=k*np.eye(fk.model.nq), dissipation=d*np.eye(fk.model.nq), attractor=config_attractor, embedding=e, dt=0.01)

# Compute joint trajectories

In [ ]:
q = np.array([a * np.pi/180 for a in config['initial_configuration']])
dq = np.array(config['initial_velocities'])
ds.x_logger = []
frequency = 0
for i in range(0, 1000):
    start = time.time()
    q, dq = ds(q, dq, kappa=0.15)
    frequency += 1/(time.time() - start)
print(f'average frequency : {frequency/1000}')

In [ ]:
ps = np.stack(ds.x_logger)

fig = plt.figure()
ax = fig.add_subplot()
for i in range(ps.shape[1]):
    ax.plot(range(ps.shape[0]), ps[:, i], label=rf'$q_{i+1}$')
plt.legend()
plt.show()

# Task space result

In [ ]:
from matplotlib import animation

def animate_robot(i, configurations, fk: ForwardKinematic, obstacle, ax):
    ax.set_aspect('equal')
    ax.set_xlim(-0.1, 2)
    ax.clear()
    ax.scatter(obstacle[:, 0], obstacle[:, 1], obstacle[:, 2], c='black', s=100)
    q = configurations[i]
    pin.forwardKinematics(fk.model, fk.data, q)
    pin.updateFramePlacements(fk.model, fk.data)
    _ = fk(q=q, dq=np.zeros_like(q), derivation_order=0)
    ax.scatter(*(fk.mus[:, i] for i in range(3)))
    for i in range(fk.sigmas.shape[0]):
        ax.set_xlim(-0.5, 0.5)
        ax.set_ylim(-0.5, 0.5)
        ax.set_zlim(-0.1, 1)
        ax.set_aspect('equal')
        plot_3d_ellipsoid_from_covariance(fk.sigmas[i], center=fk.mus[i], ax=ax)

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.view_init(azim=-90, elev=0)
dt = 0.004
step = int(np.round(0.016/dt))
ani = animation.FuncAnimation(fig, animate_robot, frames=np.arange(0, ps.shape[0], step, dtype=np.intc),fargs=(ps, fk, x, ax), interval=step*dt*1000, repeat=False)